<a href="https://colab.research.google.com/github/adelride/Django/blob/master/Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 초기 설정

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Mecab 설치로 준비를 마침

In [0]:
cd /content/gdrive/My Drive/data

/content/gdrive/My Drive/data


In [0]:
#! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [0]:
ls

Bidirectional_LSTM/         my_test_data.csv   submission_제출양식.csv
epoch1/                     my_train_data.csv  train.csv
Mecab-ko-for-Google-Colab/  public_test.csv    Word2vec.model


In [0]:
cd Mecab-ko-for-Google-Colab/

/content/gdrive/My Drive/data/Mecab-ko-for-Google-Colab


In [0]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 155kB/s 
     |████████████████████████████████| 92kB 11.0MB/s 
     |████████████████████████████████| 491kB 51.2MB/s 
  Created wheel for JPype1: filename=JPype1-0.7.1-cp36-cp36m-linux_x86_64.whl size=2453017 sha256=111cb18f44e18b60d52a0c1f868374254fe7fbd88852c571001af8469b6dc61b
  Stored in directory: /root/.cache/pip/wheels/b0/49/6a/4393ef8542c00becf80691bd242693db9e263d6e499323a984
Successfully built JPype1
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2019-12-20 04:13:53--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket

## 0. Import 그럭저럭

In [0]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

초기 필요한 Package 설치를 마침

In [0]:
import konlpy
from konlpy.tag import Mecab

tokenizer = Mecab() # setting tokenizer using Mecab()

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 1. 데이터 준비

In [0]:
cd /content/gdrive/My Drive/data

/content/gdrive/My Drive/data


In [0]:
data = pd.read_csv("train.csv")
data.head(5)
data = data[0:1000]

데이터를 Training set 과 Test set 으로 분리한다.

In [0]:
data = data.drop('year_month', axis=1)

In [0]:
# 데이터셋 smishing 비율
normal_data = data[data['smishing'] == 0]
abnormal_data = data[data['smishing'] == 1]

print("전체 건수", len(data))
print("정상 문자 건수", len(normal_data))
print("스미싱 문자 건수", len(abnormal_data))
print("스미싱 문자 비율", len(abnormal_data)/len(data)*100, "%")

전체 건수 1000
정상 문자 건수 910
스미싱 문자 건수 90
스미싱 문자 비율 9.0 %


In [0]:
# training set을 7대 3으로 나누어 7은 학습용, 3은 검증용으로 사용
# 스미싱 비율을 유지하며 샘플링
normal_sample = normal_data.sample(frac=0.7, random_state=12345) # random_state : 언제 뽑더라도 결과 동일하게 나옴
abnormal_sample = abnormal_data.sample(frac=0.7, random_state=12345) 

train_data = pd.concat([normal_sample, abnormal_sample], axis=0)
test_data = data.drop(train_data.index)

train_data.to_csv('my_train_data.csv', index=False, header=True) # 세션 끊김 방지용 내보내기
test_data.to_csv('my_test_data.csv', index=False, header=True)

In [0]:
stopwords = ['XXX', '.', '을', '를', '이', '가', '-', '(', ')', ':', '!', '?', ')-', '.-', 'ㅡ', 'XXXXXX', '..', '.(', '은', '는'] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

Mecab 을 이용하여 Token 진행

In [0]:
train_data = pd.read_csv('my_train_data.csv')
train_tokens = [[token for token, tag in tokenizer.pos(text)] for text in train_data['text']]
test_data = pd.read_csv('my_test_data.csv')
test_tokens = [[token for token, tag in tokenizer.pos(text)] for text in test_data['text']]

In [0]:
train_tokens[0:2]

[['XXX',
  '고객',
  '님',
  '소중',
  '한',
  '거래',
  '감사',
  '드립니다',
  '.',
  '행복',
  '한',
  '하루',
  '보내',
  '세요',
  '.',
  'XXX',
  '중',
  '곡동',
  'XXX',
  '올림'],
 ['XXX',
  '고객',
  '님',
  '안녕',
  '하',
  '세요',
  '?',
  '1',
  '차',
  '안',
  '내',
  '드린',
  '바',
  '와',
  '같이',
  'XXX',
  '서초',
  '로',
  '지점',
  '은',
  '서초동',
  '종합',
  '금융',
  '센터',
  '와',
  '통합',
  '되',
  '어',
  '오늘',
  '날짜',
  '로',
  '영업',
  '종료',
  '합니다',
  '통합',
  '된',
  '서초동',
  '종합',
  '금융',
  '센터',
  '에서',
  '새로운',
  '마음가짐',
  '으로',
  '모실',
  '것',
  '을',
  '약속',
  '드립니다',
  '감사',
  '합니다',
  '행복',
  '한',
  '주말',
  '보내',
  '세요',
  'XXX',
  '서초',
  '로',
  '지점',
  'XXX',
  '팀장',
  '올림']]

### Word2Vec 으로 벡터사전 생성

In [0]:
#data = pd.read_csv('train.csv')
#tokens = [[token for token, tag in tokenizer.pos(text)] for text in data['text']]

In [0]:
import gensim

In [0]:
#model = gensim.models.Word2Vec(size=300, window=2, sg = 1, alpha=0.025, min_count=100)
#model.build_vocab(tokens)
#model.train(tokens, epochs=model.iter, total_examples=model.corpus_count)

In [0]:
#model.save('Word2vec.model')

## Bi LSTM 학습

## word2vec 함수 정의

In [0]:
class Word2Vec():
    
    def __init__(self):
        None

    def tokenize(self, doc):
        tokenizer = Mecab()
        return ['/'.join(t) for t in tokenizer.pos(doc)]
    
    def read_data(self, filename):
        with open(filename, 'r',encoding='utf-8') as f:
            data = [line.split(',') for line in f.read().splitlines()]
            data = data[1:]
        return data  
    
    def Word2vec_model(self, model_name):
        
        model = gensim.models.word2vec.Word2Vec.load(model_name)
        return model
    
    def Convert2Vec(self, model_name, doc):  ## Convert corpus into vectors
        word_vec = []
        model = gensim.models.word2vec.Word2Vec.load(model_name)
        for sent in doc:
            sub = []
            for word in sent:
                if(word in model.wv.vocab):
                    sub.append(model.wv[word])
                else:
                    sub.append(np.random.uniform(-0.25,0.25,300)) ## used for OOV words
            word_vec.append(sub)
        
        return np.array(word_vec)
    
    def Zero_padding(self, train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
        zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
        for i in range(Batch_size):
            zero_pad[i,:np.shape(train_batch_X[i])[0],:np.shape(train_batch_X[i])[1]] = train_batch_X[i]
            
        return zero_pad
    
    def One_hot(self, data):
       
        index_dict = {value:index for index,value in enumerate(set(data))}
        result = []
        
        for value in data:
            
            one_hot = np.zeros(len(index_dict))
            index = index_dict[value]
            one_hot[index] = 1
            result.append(one_hot)
        
        return np.array(result)

## BI LSTM 함수 정의

In [0]:
class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
            
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_bw_cell, output_keep_prob = keep_prob)
        
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,dtype=tf.float32,
                                                                            inputs = X, sequence_length = seq_len)
        ## concat fw, bw final states
        outputs = tf.concat([states[0][1], states[1][1]], axis=1)
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # Adam Optimizer
            
        return loss, optimizer
    
    def graph_build(self):
        
        self.loss = tf.placeholder(tf.float32)
        self.acc = tf.placeholder(tf.float32)
        tf.summary.scalar('Loss', self.loss)
        tf.summary.scalar('Accuracy', self.acc)
        merged = tf.summary.merge_all()
        return merged

In [0]:
import time
import os
import tensorflow as tf
import numpy as np
import pandas as pd

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

##Tokenizer!

In [0]:
W2V = Word2Vec()

In [0]:
train_X = train_tokens
train_Y = train_data['smishing']
test_X = test_tokens
test_Y = test_data['smishing']

In [0]:
train_Y_ = W2V.One_hot(train_Y)  ## Convert to One-hot
train_X_ = W2V.Convert2Vec("Word2vec.model",train_X)  ## import word2vec model where you have trained before
test_Y_ = W2V.One_hot(test_Y)  ## Convert to One-hot
test_X_ = W2V.Convert2Vec("Word2vec.model",test_X)  ## import word2vec model where you have trained before

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
train_X_[0:2]

array([list([array([ 0.04136432, -0.27900675, -0.00306641, -0.10745669,  0.24760732,
        0.08401647, -0.10739689, -0.2759452 , -0.17938557, -0.1068137 ,
        0.02454676, -0.01855916, -0.00334774,  0.06421433,  0.14330472,
       -0.11923344,  0.09545002, -0.1789324 ,  0.05191918, -0.00351207,
        0.22260024,  0.17698865,  0.15752704, -0.04390081, -0.08677942,
        0.26399958, -0.30786145, -0.05454002,  0.01927595,  0.2440728 ,
       -0.40031183,  0.28821167,  0.02905121,  0.45718163,  0.04024874,
       -0.1394454 ,  0.12743327, -0.10608724, -0.01104242,  0.10951312,
        0.1038854 ,  0.46218616, -0.08595283, -0.1686406 , -0.12394771,
        0.5229335 , -0.24960013, -0.18405959,  0.09849028, -0.15052126,
       -0.16710265,  0.0074619 ,  0.14115685, -0.25914648,  0.3877354 ,
        0.20317073,  0.23020793, -0.00591728,  0.07087914,  0.06704313,
        0.02507903, -0.10589184, -0.11703894,  0.23098071,  0.0603651 ,
        0.24737164, -0.114788  ,  0.16658872, -0.21

In [0]:
Batch_size = 32
Total_size = len(train_X)
Vector_size = 300

#리얼 설정
train_seq_length = [len(x) for x in train_X]
test_seq_length = [len(x) for x in test_X]

Maxseq_length = max(train_seq_length) ## 95
learning_rate = 0.001
lstm_units = 128
num_class = 2
training_epochs = 4

In [0]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])
keep_prob = tf.placeholder(tf.float32, shape = None)

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

total_batch = int(len(train_X) / Batch_size)
test_batch = int(len(test_X) / Batch_size)

print("Start training!")

modelName = "./Bidirectional_LSTM/BiLSTM_model_1218.ckpt"
saver = tf.train.Saver()

train_acc = []
train_loss = []
test_acc = []
test_loss = []

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
St

In [0]:
Maxseq_length

569

In [0]:
with tf.Session(config = config) as sess:

    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter('./Bidirectional_LSTM', sess.graph)
    merged = BiLSTM.graph_build()
    
    for epoch in range(training_epochs):

        avg_acc, avg_loss = 0. , 0.
        mask = np.random.permutation(len(train_X_))
        train_X_ = train_X_[mask]
        train_Y_ = train_Y_[mask]
        
        for step in range(total_batch):

            train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size]
            train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size]
            batch_seq_length = train_seq_length[step*Batch_size : step*Batch_size+Batch_size]
            
            train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
            
            sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            # Compute average loss
            loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                              keep_prob : 0.75})
            avg_loss += loss_ / total_batch
            
            acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                                 keep_prob : 0.75})
            avg_acc += acc / total_batch
            print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))
   
        summary = sess.run(merged, feed_dict = {BiLSTM.loss : avg_loss, BiLSTM.acc : avg_acc})       
        train_writer.add_summary(summary, epoch)
    
        t_avg_acc, t_avg_loss = 0. , 0.
        print("Test cases, could take few minutes")
        for step in range(test_batch):
            
            test_batch_X = test_X_[step*Batch_size : step*Batch_size+Batch_size]
            test_batch_Y = test_Y_[step*Batch_size : step*Batch_size+Batch_size]
            batch_seq_length = test_seq_length[step*Batch_size : step*Batch_size+Batch_size]
            
            test_batch_X = W2V.Zero_padding(test_batch_X, Batch_size, Maxseq_length, Vector_size)
            
            # Compute average loss
            loss2 = sess.run(loss, feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                              keep_prob : 1.0})
            t_avg_loss += loss2 / test_batch
            
            t_acc = sess.run(accuracy , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                                   keep_prob : 1.0})
            t_avg_acc += t_acc / test_batch

        print("<Train> Loss = {:.6f} Accuracy = {:.6f}".format(avg_loss, avg_acc))
        print("<Test> Loss = {:.6f} Accuracy = {:.6f}".format(t_avg_loss, t_avg_acc))
        train_loss.append(avg_loss)
        train_acc.append(avg_acc)
        test_loss.append(t_avg_loss)
        test_acc.append(t_avg_acc)

    train_loss = pd.DataFrame({"train_loss":train_loss})
    train_acc = pd.DataFrame({"train_acc":train_acc})
    test_loss = pd.DataFrame({"test_loss":test_loss})
    test_acc = pd.DataFrame({"test_acc":test_acc})
    df = pd.concat([train_loss,train_acc,test_loss,test_acc], axis = 1)
    df.to_csv("./Bidirectional_LSTM/loss_accuracy.csv", sep =",", index=False)
    
    train_writer.close()
    duration = time.time() - start_time
    minute = int(duration / 60)
    second = int(duration) % 60
    print("%dminutes %dseconds" % (minute,second))
    save_path = saver.save(sess, modelName)
    
    print ('save_path',save_path)

epoch : 01 step : 0001 loss = 0.488529 accuracy= 0.906250
epoch : 01 step : 0002 loss = 0.415778 accuracy= 0.875000
epoch : 01 step : 0003 loss = 0.298770 accuracy= 0.937500
epoch : 01 step : 0004 loss = 0.308595 accuracy= 0.906250
epoch : 01 step : 0005 loss = 0.061324 accuracy= 1.000000
epoch : 01 step : 0006 loss = 0.382105 accuracy= 0.875000
epoch : 01 step : 0007 loss = 0.202378 accuracy= 0.937500
epoch : 01 step : 0008 loss = 0.358133 accuracy= 0.875000
epoch : 01 step : 0009 loss = 0.085467 accuracy= 0.968750
epoch : 01 step : 0010 loss = 0.141361 accuracy= 0.906250
epoch : 01 step : 0011 loss = 0.251100 accuracy= 0.906250
epoch : 01 step : 0012 loss = 0.093763 accuracy= 1.000000
epoch : 01 step : 0013 loss = 0.098053 accuracy= 0.968750
epoch : 01 step : 0014 loss = 0.096033 accuracy= 0.968750
epoch : 01 step : 0015 loss = 0.129818 accuracy= 0.937500
epoch : 01 step : 0016 loss = 0.096798 accuracy= 0.968750
epoch : 01 step : 0017 loss = 0.187965 accuracy= 0.875000
epoch : 01 ste

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
save_path = saver.save(sess, BiLSTM.ckpt)

AttributeError: ignored